# GPU Metrics Utilization Summary

This recipe provides a summary of different GPU metrics for up to REPLACE_N longest ranges, as specified by the `--longest-n` option.

In [ ]:
import pandas as pd
import plotly.offline as pyo

from IPython.display import display, HTML, Markdown

import nsys_display

display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.1f}'.format
pyo.init_notebook_mode()

## Per-rank statistics

The table shows the GPU metrics statistics for the selected drop-down values. Note that:
* All time values are in nanoseconds.
* 'Q1', 'Med', and 'Q3' are the 25th, 50th, and 75th percentiles of the data set, respectively.
* If the `--per-*` option was used, the statistics represent an aggregation of the statistics generated per range instance. In this case, 'Q1', 'Med', and 'Q3' are approximated as the minimum of Q1, the median of medians, and the maximum of Q3 for each unique range name.

In [ ]:
stats_df = pd.read_parquet("rank_stats.parquet")

group_by_columns = ["Rank", "GPU ID"]
if "Name" in stats_df.index.names:
    group_by_columns += ["Name"]

nsys_display.display_stats_per_column(stats_df, group_by_columns)

## Top and bottom ranges per rank

The bar charts below display the top and bottom N ranges, either for the projected NVTX range or the kernel, based on the selected `--per-*` option. Ranges with a value of 0 will not be shown in the chart.

Use the slider to adjust the value of N.

In [ ]:
if "Name" not in stats_df.index.names:
    display(Markdown("No range data available. Please use one of the `--per` options to display this graph."))
else:
    nsys_display.display_top_bottom_n(stats_df, ["Rank", "GPU ID", "Metric Name"], "Name", "Mean", xaxis_title="Range", yaxis_title="Value", title="Average metric value")

## Files

The table associates each rank number with the original filename. Ranks are assigned assuming that the file names include the rank with sufficient zero padding for proper sorting. Otherwise, the actual rank may differ from the assigned ID.

In [ ]:
files_df = pd.read_parquet("files.parquet")
display(files_df)